In [18]:
# Load dependencies
import pickle
import PIL
from PIL import UnidentifiedImageError
import tensorflow as tf
from tensorflow import keras
from keras.applications.vgg16 import preprocess_input
from keras.callbacks import ModelCheckpoint
from os.path import isdir
from os import scandir
from os import remove

In [19]:
# Filter out corrupted images
imgs_deleted = 0

# Loop through all images of each class
for subdir in scandir("./PetImages"):
    # If item is a directory, loop through all its contents
    if isdir(subdir):
        for img in scandir(subdir):
            # Try to open image
            # If unable, assume img is corrupted and remove it
            try:
                img = PIL.Image.open(img.path)
            except PIL.UnidentifiedImageError:
                remove(img.path)
                imgs_deleted += 1
                
print(imgs_deleted, "images deleted.")

4 images deleted.


In [20]:
# Create the data generator
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2,
)

In [21]:
# Create training set as generator
train_generator = train_datagen.flow_from_directory(
    "./PetImages",
    target_size=(224,224),
    batch_size=32,
    shuffle=True,
    subset='training'
)

Found 20000 images belonging to 2 classes.


In [22]:
# Create validation set as generator
val_generator = train_datagen.flow_from_directory(
    "./PetImages",
    target_size=(224,224),
    batch_size=32,
    shuffle=True,
    subset='validation'
)

Found 4998 images belonging to 2 classes.


In [23]:
# Build model by creating instance of VGG16
model = keras.applications.VGG16(
    include_top=True,
    weights=None,
    classes=2,
    classifier_activation='softmax'
)

In [24]:
# Print model details
model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [25]:
# Specify optimizer, loss, and metrics for model
model.compile(
    optimizer='rmsprop', 
    loss='categorical_crossentropy',
    metrics = ['accuracy']
)

In [26]:
# Specify callbacks
model_checkpoint_callback = ModelCheckpoint(
    "./models/checkpoints/cats-and-dogs/save_at_epoch_{epoch}.h5",
    monitor="val_loss",
    save_best_only=True,
)

In [27]:
# Fit data to model
history = model.fit(
    train_generator, 
    batch_size=32, 
    epochs=1,
    callbacks=[model_checkpoint_callback],
    validation_data=val_generator
)

2022-04-18 16:04:42.052438: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


625/625 [==============================] - ETA: 0s - loss: 349444.1562 - accuracy: 0.4998

2022-04-18 16:43:55.967781: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


625/625 [==============================] - 2447s 4s/step - loss: 349444.1562 - accuracy: 0.4998 - val_loss: 0.6932 - val_accuracy: 0.5000


In [28]:
# View training loss and metrics
print(history.history)

{'loss': [349444.15625], 'accuracy': [0.49980002641677856], 'val_loss': [0.6931754946708679], 'val_accuracy': [0.5]}


In [29]:
# Save training history as binary file using pickle
with open("./models/data/training_history_cats_and_dogs", "wb") as training_history:
    pickle.dump(history.history, training_history)